# Lab 5 Threat hunting

## Diego Franco 20240

In [67]:
import pandas as pd
from pandas import json_normalize
import json
import random
from clasificador import *
import csv
import whois

### Leer el json con la data

In [68]:
import json

# Paso 1: Definir la ubicación del archivo
archivo = "Archivos/large_eve.json"

# Paso 2: Cargar el archivo JSON en una lista de diccionarios
with open(archivo, 'r') as f:
    data = [json.loads(line) for line in f]

# Paso 3: Obtener la cantidad total de registros
cantidad_total_registros = len(data)
print("Cantidad total de registros:", cantidad_total_registros)



Cantidad total de registros: 746909


### Obtener unicamente los DNS

In [69]:
# Filtrar la lista de diccionarios para obtener registros DNS
registros_dns = [registro for registro in data if 'event_type' in registro and 'dns' in registro['event_type'].lower()]

# Obtener la cantidad de registros DNS
cantidad_registros_dns = len(registros_dns)
print("Cantidad de registros DNS:", cantidad_registros_dns)



Cantidad de registros DNS: 15749


### Mostrar 2 registros

In [70]:
# Seleccionar dos registros aleatorios de registros_dns
dos_registros_cualesquiera = random.sample(registros_dns, 2)

# Imprimir los dos registros seleccionados
for registro in dos_registros_cualesquiera:
    print(registro)

{'timestamp': '2017-07-22T18:29:19.033480-0500', 'flow_id': 710973884170952, 'pcap_cnt': 507339, 'event_type': 'dns', 'vlan': 150, 'src_ip': '192.168.205.188', 'src_port': 44875, 'dest_ip': '192.168.207.4', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 5226, 'rrname': 'mirrors.rit.edu', 'rrtype': 'AAAA', 'tx_id': 0}}
{'timestamp': '2017-07-22T19:29:19.438563-0500', 'flow_id': 1792436147826706, 'pcap_cnt': 3355561, 'event_type': 'dns', 'vlan': 150, 'src_ip': '192.168.207.4', 'src_port': 53, 'dest_ip': '192.168.205.188', 'dest_port': 57922, 'proto': 'UDP', 'dns': {'type': 'answer', 'id': 33139, 'rcode': 'NXDOMAIN', 'rrname': '<root>', 'rrtype': 'SOA', 'ttl': 20864}}


### Convertir a data frame

In [71]:
# Normalizar la información contenida en los JSON anidados y asignarla a un DataFrame
df_normalized = json_normalize(registros_dns)

# Mostrar la forma (shape) del DataFrame
print("Shape del DataFrame:", df_normalized.shape)

Shape del DataFrame: (15749, 18)


In [72]:
# Filtrar los registros DNS tipo A
registros_dns_tipo_a = df_normalized[df_normalized['dns.rrtype'] == 'A']

print("Shape del DataFrame con registros DNS tipo A:", registros_dns_tipo_a.shape)

Shape del DataFrame con registros DNS tipo A: (2849, 18)


### Filtrar dominios unicos

In [73]:
dominios_unicos = registros_dns_tipo_a['dns.rrname'].unique()
print("Cantidad de dominios únicos:", len(dominios_unicos))

Cantidad de dominios únicos: 177


### TLD

In [74]:
def obtener_tld(dominio):
    """
    Función para obtener el TLD (Dominio de Nivel Superior) de un dominio dado.
    
    Args:
    - dominio (str): El dominio del cual se quiere obtener el TLD.
    
    Returns:
    - str: El TLD del dominio.
    """
    partes = dominio.split('.')
    for i in range(len(partes) - 1, 0, -1):
        # Recorrer las partes del dominio en orden inverso
        tld_potencial = '.'.join(partes[i:])
        if tld_potencial in ["com", "net", "org", "gov", "edu", "mil", "arpa", "int"]:
            continue  # Si el segmento es un TLD válido, continuar buscando
        else:
            return tld_potencial  # Si no es un TLD válido, devolver este segmento como el TLD
    return dominio

In [75]:
# Aplicar la función obtener_tld a cada dominio único
tlds = [obtener_tld(dominio) for dominio in dominios_unicos]

# Crear un nuevo DataFrame solo con la columna domain_tld
df_tlds = pd.DataFrame({'domain_tld': tlds})

print(df_tlds)

           domain_tld
0    wunderground.com
1         dropbox.com
2           aoltw.net
3                home
4         mozilla.com
..                ...
172       localdomain
173     microsoft.com
174      verisign.com
175    stayonline.net
176          real.com

[177 rows x 1 columns]


### Utilizar el clasificador

In [76]:
df_resultado = clasificacion(df_tlds)

print(df_resultado)

           domain_tld  isDGA
0    wunderground.com      0
1         dropbox.com      1
2           aoltw.net      1
3                home      0
4         mozilla.com      0
..                ...    ...
172       localdomain      0
173     microsoft.com      1
174      verisign.com      0
175    stayonline.net      0
176          real.com      1

[177 rows x 2 columns]


c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [77]:

df_dga = df_resultado[df_resultado['isDGA'] == 1]

df_dga_unicos = df_dga.drop_duplicates()

print(len(df_dga_unicos))


32


### Filtrar con la lista

In [78]:
def verificar_tld(tld):
    """
    Función para verificar si un TLD se encuentra en una lista de TLDs proporcionada en un archivo CSV.

    Args:
    - tld (str): El TLD que se desea verificar.

    Returns:
    - int: 0 si el TLD está en la lista, 1 si no está.
    """
    # Ruta al archivo CSV que contiene la lista de TLDs
    archivo_tlds = 'top-1m.csv'

    # Intentamos abrir el archivo y verificar si el TLD está en la lista
    try:
        with open(archivo_tlds, 'r') as f:
            for linea in f:
                if linea.strip().endswith(tld.lower()):
                    return 0  # El TLD está en la lista
            return 1  # El TLD no está en la lista
    except FileNotFoundError:
        print("Error: No se pudo encontrar el archivo de lista de TLDs.")
        return -1 

In [79]:
# Crear una copia del DataFrame df_dga_unicos
df_dga_unicos_copia = df_dga_unicos.copy()

df_dga_unicos_copia['en_lista'] = df_dga_unicos_copia['domain_tld'].apply(verificar_tld)

df_en_lista = df_dga_unicos_copia[df_dga_unicos_copia['en_lista'] == 1]

print(len(df_en_lista))

df_en_lista_sin_duplicados = df_en_lista.drop_duplicates(subset='domain_tld')

print(df_en_lista_sin_duplicados)



10
               domain_tld  isDGA  en_lista
2               aoltw.net      1         1
18          110phpmyadmin      1         1
34         sql-ledger.org      1         1
35    backtrack-linux.org      1         1
64      bigflickrfeed.com      1         1
82        malwarecity.com      1         1
89            cakephp.org      1         1
97              ecvps.com      1         1
160  vtlfccmfxlkgifuf.com      1         1
162        macromates.com      1         1


In [80]:
import whois
from datetime import datetime

def obtener_fecha_creacion_tld(tld):

    try:
        w = whois.whois(tld)

        fecha_creacion = w.creation_date

        if isinstance(fecha_creacion, list):
            fecha_creacion = fecha_creacion[0]

        if fecha_creacion:
            if isinstance(fecha_creacion, datetime):
                return fecha_creacion.strftime('%Y-%m-%d')
            else:
                return str(fecha_creacion)
        else:
            return "No se encontro."
    except Exception as e:
        return "No se encontro. "

In [81]:
df_en_lista_sin_duplicados['fecha_creacion'] = df_en_lista_sin_duplicados['domain_tld'].apply(obtener_fecha_creacion_tld)

# Mostrar el DataFrame con la fecha de creación
print(df_en_lista_sin_duplicados)

               domain_tld  isDGA  en_lista    fecha_creacion
2               aoltw.net      1         1        2000-01-10
18          110phpmyadmin      1         1   No se encontro.
34         sql-ledger.org      1         1        2000-09-08
35    backtrack-linux.org      1         1        2009-04-29
64      bigflickrfeed.com      1         1  No se encontro. 
82        malwarecity.com      1         1        2008-02-06
89            cakephp.org      1         1        2005-06-13
97              ecvps.com      1         1        2009-05-21
160  vtlfccmfxlkgifuf.com      1         1  No se encontro. 
162        macromates.com      1         1        2003-02-05


### Dominios sospechosos

Se puede confirmar como dominio sospechoso:


In [82]:
# Filtrar las filas donde la fecha de creación es mayor al año 2006
filtro_fecha = df_en_lista_sin_duplicados['fecha_creacion'] > '2006-01-01'
df_filtrado = df_en_lista_sin_duplicados[filtro_fecha]

print(df_filtrado)

               domain_tld  isDGA  en_lista    fecha_creacion
18          110phpmyadmin      1         1   No se encontro.
35    backtrack-linux.org      1         1        2009-04-29
64      bigflickrfeed.com      1         1  No se encontro. 
82        malwarecity.com      1         1        2008-02-06
97              ecvps.com      1         1        2009-05-21
160  vtlfccmfxlkgifuf.com      1         1  No se encontro. 


Estos dominos se pueden considerar como sospechosos porque son los dominios mas recientes y ademas son los que no se le pudieron encontar una fecha de creacion, esto se debe a que: 

- Los dominios más antiguos tienden a tener una reputación establecida y pueden haber sido utilizados por un tiempo considerablemente largo. 

- La ausencia de información sobre la fecha de creación del dominio puede percibirse como una falta de transparencia por parte del propietario del dominio.

### Dominos DGA con patron aleatorio

Tras hacer la revision de los dominios para detectar cual de ellos posee una secuencias aleatorias de caracteres, no palabras y que termina de indicar que son los dominios más sospechosos de todo el listado, se puede concluir que los dominios que cumplen con estas características son:
- ecvps.com      
- vtlfccmfxlkgifuf.com 

### Prompt utilizados

- Necesito una función que obtenga el TLD para un dominio. Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para safebrowsing.clients.google.com.home, el TLD es home.

- Ahora necesito una función que utilice la lista de un millón de TLD que se encuentra en mi directorio top-1m.csv, y devuelva 0 si el TLD se encuentra en la lista y 1 si no está. 
